# 05 — Export TFLite (optionnel)

Exporter le modèle entraîné en TFLite (float32 ou quantifié int8).

In [ ]:
import os

TF_MODELS_DIR = "models"
PIPELINE_CONFIG_PATH = "training/ssd_mobilenet_v3_small_fpnlite_320x320_coco17_tpu-8/pipeline.config"
MODEL_DIR = "checkpoints/ssd_mobilenet_v3_small_frelon"
EXPORT_DIR = "exported_models/ssd_mobilenet_v3_small_frelon"

os.makedirs(EXPORT_DIR, exist_ok=True)

print(f"""
Exporter en SavedModel:

python {TF_MODELS_DIR}/research/object_detection/exporter_main_v2.py \
  --input_type=image_tensor \
  --pipeline_config_path={PIPELINE_CONFIG_PATH} \
  --trained_checkpoint_dir={MODEL_DIR} \
  --output_directory={EXPORT_DIR}
""")

print(f"""
Puis conversion en TFLite:

python - <<'PY'
import tensorflow as tf
import pathlib
saved_model_dir = r"{EXPORT_DIR}/saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
pathlib.Path("{EXPORT_DIR}/model.tflite").write_bytes(tflite_model)
print("TFLite exporté vers {EXPORT_DIR}/model.tflite")
PY
""")

print(f"""
Quantification int8 (post-training) — exemple minimal:

python - <<'PY'
import tensorflow as tf, pathlib
saved_model_dir = r"{EXPORT_DIR}/saved_model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Optionnel: jeu de calibration (recommandé)
# converter.representative_dataset = ...
# Pour garantir int8 pur (Edge TPU, microctrl.), définir les types:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8
# converter.inference_output_type = tf.int8
quant_tflite = converter.convert()
pathlib.Path("{EXPORT_DIR}/model_int8.tflite").write_bytes(quant_tflite)
print("TFLite int8 exporté vers {EXPORT_DIR}/model_int8.tflite")
PY
""")